In [ ]:
# This cell loads libraries that we need
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import matplotlib as mpltlb

## Network metrics using larval connectivity graphs
We're going to start by importing a connectivity matrix for reefs in the central Great Barrier Reef. This data was generated using a high-resolution hydrodynamic model of water flow in the GBR lagoon, for 1998 (we chose this year because it experienced the first really large mass coral bleaching of the modern era).

Coral reef fishes generally spend their whole lives on a single reef. However, every so often (e.g., annually) they take part in broadcast spawning events where fertilised eggs are released into the pelagic environment. This exchange is critical for understanding the ecological and evolutionary dynamics of the reef.

In the code that follows, we're going to load data on this process, and use it to calculate and interpret a couple of network metrics.

### Graph nodes
By graph analogy, each of the patch reefs in the GBR are nodes in a network. There are 320 reefs being modelled, and therefore 320 nodes. 

We are going to import the latitude and longitude of each of these nodes, and plot them on a map of Australia.


In [ ]:
# load data on the latitude and longitude of the centroids of the GBR reefs
url = 'https://raw.githubusercontent.com/MikeBode/SAEF_quantitative_skills/main/centroid.csv'
centroid = pd.read_csv(url, error_bad_lines=False, header=None)
print(centroid)
centroid = centroid.to_numpy()

Now we need to load data that will allow us to make a map of Australia.

In [ ]:
# load data on the outline of the Australian coastline. We'll need this for later
url = 'https://raw.githubusercontent.com/MikeBode/SAEF_quantitative_skills/main/AustOutline.csv'
AustOutline = pd.read_csv(url, error_bad_lines=False, header=None)

# convert this dataframe object to a numPy array so the Path command works
AustOutNP = AustOutline.to_numpy()

# plot Australian outline using the patch command
# plot the reef locations as dots in top
from matplotlib.path import Path
from matplotlib.patches import PathPatch

# Set up a figure called "ax"
fig, ax = plt.subplots()

# Plot the outline of Australia as a long squiggly line
ax.plot(AustOutline.iloc[:, 0], AustOutline.iloc[:, 1])
path = Path(AustOutNP)

# Fill the line in with a simple color
patch = PathPatch(path, facecolor='C0', edgecolor='none', alpha=0.3)
ax.add_patch(patch)
ax.set_aspect('equal', 'box')

# plot the locations of all the reefs as dots
ax.plot(centroid[:, 0], centroid[:, 1], 'k.', markersize = 5)
plt.xlim([144.4,147.7]) # might need to do axs.xlim but idk
plt.ylim([-19.25,-13.6])
ax.set_aspect('equal', 'box')
plt.rcParams["figure.figsize"] = (10, 7)

plt.show()

### Graph edges

In our graphical analogy of the GBR's reef fish populations, each reef is joined by larval connectivity (demographic exchange of baby fishes) that create edges between the nodes.

Each of the 320 nodes can be connected to any of the other 320 nodes, including themselves (a larvae can spend time floating in the GBR lagoon and end up returning to its natal reef). There are therefore $320 \times 320$ possible edges - more than one million.

We're going to load this edge data as a "connectivity matrix". This is a $320 \times 320$ matrix where each element measures the probability of a given edge. For example, on row 5, column 7, we have the element $p_{5,7}$. This is the probability that a larvae leaving from reef 5 is going to land safely on reef 7. 

### Visualising the graph as a matrix
Let's load this whole matrix and have a look at it.

In [ ]:
# load data on a connectivity network using Pandas
url = 'https://raw.githubusercontent.com/MikeBode/SAEF_quantitative_skills/main/ConnMat.csv'
C = pd.read_csv(url, error_bad_lines=False, header=None)

# Show what the data looks like
print(C)

# need to convert C to a numpy array, which is easier to deal with
C = C.to_numpy()


This matrix is too large to look at as text. Let's visualise the structure of the edges as a sparse matrix image. This type of image shows nonzero elements as coloured, and zero elements as white. It doesn't tell us how strong the connections are, but it does give us an idea of the structure.

In [ ]:
# create a new figure
fig, axs = plt.subplots()

# The command "spy" produces a visualisation of nonzero elements in a matrix
plt.spy(C, markersize=1)
axs.set_xlabel('Destination reef',fontsize = 18);
axs.set_ylabel('Source reef', fontsize = 18);
plt.rcParams["figure.figsize"] = (7,7)
plt.show()

The first thing we see is that the graph is both dense and sparse. There are lots of connections that don't exist (the white space), and lots that do (the blue parts). We can also quickly see that most of the strong connections are near the diagonal of the matrix - near the line extending from the top left hand of the matrix to the bottom right hand. Of course, not all these elements are *on* the diagonal - most are just near it. That's because larvae tend to be transported to reefs that are nearby their natal reef. The further away we get from the natal reef, the weaker the connections, *on average*. 

The second thing we notice is the strange white space in the lower left quadrant of the matrix. Looking carefully, there are almost no connections coming out of reefs 200-320, and going into reefs 0-200. The opposite is not true. There are plenty of connections coming out of reefs 0-200 and ending in reefs 200-300. Something large and asymmetric is going on in this network.


### Mapping the graph onto space
Now let's visualise this graph as connections between points in space. We're going to take our map of the reefs and the Australian coastline, and we're going to put edges (lines) in between those nodes that are strongly connected.

In [ ]:
# create a new figure
fig, axs = plt.subplots()
axs.plot(AustOutNP[:,0],AustOutNP[:,1],'k')

# plot the connections between reef centroids

# Go through every source reef in a loop
for i in range(320):

  # Now go through every destination reef with a NESTED loop
  for j in range(320):

    # We're going to plot just the reefs in the southern section of our model.
    # That is, those further south than 18 degrees.
    if centroid[i, 1] < -18 and centroid[j, 1] < -18:

      ### SET THIS VALUE TO GRAPH ONLY THE STRONGEST LINKS
      EdgeStrengthThreshold = 0.00001;

      # Only plot a link if it's stronger than the threshold
      if C[i, j] > EdgeStrengthThreshold:

        # This command plots a link between a source reef and a destination reef
        axs.plot(centroid[[i, j], 0], centroid[[i, j], 1], '-', color = [0.8, 0.8, 0.8])


# Plot the controids themselves. We need to plot these on top, or they will be 
# hidden by all the edges
axs.plot(centroid[:, 0], centroid[:, 1], '.', markersize = 10, color = 'm')

# Let's zoom into the map so we don't see irrelvant locations
plt.xlim([146, 147.5]) # This zooms in the x direction (longitudinally)
plt.ylim([-19, -18]) # This zooms in the y direction (latitudinally)

axs.set_aspect('equal', 'box')
plt.rcParams["figure.figsize"] = (7, 7)
plt.show()

### STOP HERE

### Calculating network metrics for the GBR dataset

We're going to start simply, by calculating the degree of all the nodes in the network. The degree is the number of edges that either enter or leave a node.

In [ ]:
# Start by creating an array to store the values in
OutDegree = np.zeros((320,1))

for i in range(320):
  for j in range(320):
    if C[i,j] > 0:
      OutDegree[i] = OutDegree[i] + 1

# Repeat this process for the in degree
InDegree = np.zeros((320,1))

for i in range(320):
  for j in range(320):
    if C[i,j] > 0:
      InDegree[j] = InDegree[j] + 1


Now let's plot the in and out degree as a colour associated with each node in the network.

In [ ]:
# map the degree of each node in space
fig, axs = plt.subplots()
FS = 18;
axs.plot(AustOutNP[:,0], AustOutNP[:,1],'k')

# split the out degrees into quantiles
Q = np.quantile(OutDegree, [0.2, 0.4, 0.6, 0.8]);
CL = np.array([[0, 0.2, 1], [0, 1, 1], [0.5, 1, 0.5], [1, 1, 0], [1, 0.1, 0]])
MS = 10

for i in range(320):
    
  # plot the centroids in a different colour based on the quintile they occupy
  if OutDegree[i] < Q[0]:
    axs.plot(centroid[i, 0], centroid[i, 1], '.', markersize = MS, color = CL[0, :])
  elif OutDegree[i] < Q[1]:
    axs.plot(centroid[i, 0], centroid[i, 1], '.', markersize = MS, color = CL[1, :])
  elif OutDegree[i] < Q[2]:
    axs.plot(centroid[i, 0], centroid[i, 1], '.', markersize = MS, color = CL[2, :])
  elif OutDegree[i] < Q[3]:
    axs.plot(centroid[i, 0], centroid[i, 1], '.', markersize = MS, color = CL[3, :])
  else:
    axs.plot(centroid[i, 0], centroid[i, 1], '.', markersize = MS, color = CL[4, :])

# split the in degrees into quantiles
Q = np.quantile(InDegree, [0.2, 0.4, 0.6, 0.8])

for i in range(320):
    
  # plot the centroids in a different colour based on the quintile they occupy
  if InDegree[i] < Q[0]:
    axs.plot(centroid[i, 0]+2, centroid[i, 1], '.', markersize = MS, color = CL[0, :])
  elif InDegree[i] < Q[1]:
    axs.plot(centroid[i, 0]+2, centroid[i, 1], '.', markersize = MS, color = CL[1, :])
  elif InDegree[i] < Q[2]:
    axs.plot(centroid[i, 0]+2, centroid[i, 1], '.', markersize = MS, color = CL[2, :])
  elif InDegree[i] < Q[3]:
    axs.plot(centroid[i, 0]+2, centroid[i, 1], '.', markersize = MS, color = CL[3, :])
  else:
    axs.plot(centroid[i, 0]+2, centroid[i, 1], '.', markersize = MS, color = CL[4, :])

plt.text(145.25, -13.75, 'Out degree', horizontalalignment = 'center', fontsize = FS)
plt.text(148.25, -13.75, 'In degree', horizontalalignment = 'center', fontsize = FS)
plt.xlim([144.4, 150])
plt.ylim([-19, -13.5])

axs.set_aspect('equal', 'box')
plt.rcParams["figure.figsize"] = (7, 7)
plt.show()


In this figure, blue markers have low degrees, and orange-red markers have high degrees. It's clear that the connectivity network at this point on the GBR has lots of outward connections in the north, and lots of inward connections in the south. This reinforces the asymmetry we saw in the network sparsity figure.

In [ ]:
# show histograms of the node degrees
fig, axs = plt.subplots(3, 1)

# first subplot
axs[0].hist(OutDegree,25);
axs[0].set_xlabel('Out degree distribution', fontsize = FS);
axs[0].set_ylabel('Frequency', fontsize = FS);

# second subplot
axs[1].hist(InDegree,25);
axs[1].set_xlabel('Out degree distribution', fontsize = FS);
axs[1].set_ylabel('Frequency', fontsize = FS);
plt.xlim([0, 320])

# last two sublots
axs[2].plot(InDegree, OutDegree, '.')
plt.rcParams["figure.figsize"] = (7, 12)
plt.show()


### STOP HERE

### Network clustering 

Now we're going to calculate another metric associated with each node - it's clustering coefficient, as defined by Watts and Strogatz (1998). By their definition, the clustering coefficient is the proportion of potential triangles associated with a node that are in fact triangles.

With graphs that are asymmetric, like connectivity on the GBR, we can treat 

In [ ]:
# make the matrix symmetric and binary
C = C + C.transpose()

# create a new binary matrix
CNew = np.zeros([320, 320])
for i in range(320):
  for j in range(320):
    if C[i, j] > 0:
      CNew[i, j] = 1

# replace the old C matrix
C = CNew

In [ ]:
# define the clustering variable
Clustering = np.zeros(320)


# go through each node
for i in range(320):

    # Clustering is the proportion of possible triangles that are actually triangles
    # We therefore need to count both
    count = 0;
    cliques = 0; 

    for j in range(320):
      for k in range(320):
        if C[i,j] > 0 and C[i,k] > 0:

          # If we're here in the loop, then i-j is a link, and i-k is a link                
          count = count + 1;

          # If j-k is also a link, then we have a triangle.
          if C[j, k] > 0:
            cliques = cliques + 1;

    # What proportion of potential triangles were actually closed triangles?
    Clustering[i] = cliques / count;

In [ ]:
# Map the clusteredness in space
fig, axs = plt.subplots()
FS = 18;

# Split the clusteredness into quintiles
Q = np.quantile(Clustering, [0.2, 0.4, 0.6, 0.8]);
CL = np.array([[0, 0.2, 1], [0, 1, 1], [0.5, 1, 0.5], [1, 1, 0], [1, 0.1, 0]])

MS = 12

for i in range(320):
    
  # plot the centroids in a different colour based on the quintile they occupy
  if Clustering[i] < Q[0]:
    axs.plot(centroid[i, 0], centroid[i, 1], '.', markersize = MS, color = CL[0, :])
  elif Clustering[i] < Q[1]:
    axs.plot(centroid[i, 0], centroid[i, 1], '.', markersize = MS, color = CL[1, :])
  elif Clustering[i] < Q[2]:
    axs.plot(centroid[i, 0], centroid[i, 1], '.', markersize = MS, color = CL[2, :])
  elif Clustering[i] < Q[3]:
    axs.plot(centroid[i, 0], centroid[i, 1], '.', markersize = MS, color = CL[3, :])
  else:
    axs.plot(centroid[i, 0], centroid[i, 1], '.', markersize = MS, color = CL[4, :])

# set up the figure
plt.xlim([144.4, 147.75])
plt.ylim([-19, -13.5])
# fig.colorbar() # need to fix up the colorbar smh

axs.plot(AustOutNP[:,0], AustOutNP[:,1],'k')

axs.set_aspect('equal', 'box')
plt.rcParams["figure.figsize"] = (10, 10)
plt.show()
